#Import Required Libraries

In [1]:
import logging
import simpy
from numpy import random

from dacdam.network import Network, Router, Server, Subnet, Sensor
from dacdam.admin import NetworkAdministrator
from dacdam.software import Vulnerability, VulnerabilityManager

#Setup Execution Process

In [2]:
random.seed(1234567890)

NUM_VULNERABILITIES = 400                  # initial number of vulnerabilities

NUM_NETWORKS = 2
NUM_SENSORS = 15
NUM_SERVERS = 12
NUM_SUBNETS = 5
NUM_ROUTERS = 10

YEARS_TO_SIMULATE = 5

env = simpy.Environment()

vulnerability_mgr = VulnerabilityManager(env=env, num_vulnerabilities=1000)

admins = []
for j in range(NUM_NETWORKS):
    network_items = [Router(env=env, name="Router_%04d" % (j*NUM_ROUTERS+i+1)) for i in range(NUM_ROUTERS)] + \
                    [Server(env=env, name="Server_%04d" % (j*NUM_SERVERS+i+1)) for i in range(NUM_SERVERS)] + \
                    [Subnet(env=env, name="Subnet_%04d" % (j*NUM_SUBNETS+i+1)) for i in range(NUM_SUBNETS)] + \
                    [Sensor(env=env, name="Sensor_%04d" % (j*NUM_SENSORS+i+1)) for i in range(NUM_SENSORS)]

    network_admin = NetworkAdministrator(env=env,
                                         name="Admin_%02d" % (j+1),
                                         patches=vulnerability_mgr.patches,
                                         vulnerabilities=vulnerability_mgr.vulnerabilities,
                                         network_items=network_items)

    for vulnerability in vulnerability_mgr.vulnerabilities.items:
        env.process(network_admin.add_vulnerability(vulnerability))
    
    admins.append(network_admin)

#Run and Profile the Simulation

In [3]:
%prun env.run(until=YEARS_TO_SIMULATE*365.25)